# Train shenanigans

In [13]:
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np
from threading import Thread

%matplotlib inline

In [14]:
df = pd.read_csv('./train_data/2021-04-01_istdaten.csv', error_bad_lines=False, sep=';')
df.head()

C:\ProgramData\Anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
0,01.04.2021,80:06____:17001:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17001,RE,NaN,RE,...,False,8500090,Basel Bad Bf,01.04.2021 06:29,01.04.2021 06:30:00,PROGNOSE,NaN,NaN,PROGNOSE,False
1,01.04.2021,80:06____:17003:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17003,RE,NaN,RE,...,False,8500090,Basel Bad Bf,01.04.2021 08:02,01.04.2021 08:04:00,PROGNOSE,NaN,NaN,PROGNOSE,False
2,01.04.2021,80:06____:17005:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17005,RE,NaN,RE,...,True,8500090,Basel Bad Bf,01.04.2021 09:29,NaN,UNBEKANNT,NaN,NaN,UNBEKANNT,False
3,01.04.2021,80:06____:17006:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17006,RE,NaN,RE,...,False,8500090,Basel Bad Bf,NaN,NaN,PROGNOSE,01.04.2021 06:11,NaN,UNBEKANNT,False
4,01.04.2021,80:06____:17007:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17007,RE,NaN,RE,...,True,8500090,Basel Bad Bf,01.04.2021 10:02,NaN,UNBEKANNT,NaN,NaN,UNBEKANNT,False


In [19]:
df[df.UMLAUF_ID.isna() == False].head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
87501,01.04.2021,85:146:256349-00795-1,85:146,STI,Verkehrsbetriebe STI AG,Bus,85:146:1,1,10.0,B,...,False,8576333,"Spiez, Bahnhof",NaN,NaN,PROGNOSE,01.04.2021 05:30,01.04.2021 05:30:00,PROGNOSE,False
87502,01.04.2021,85:146:256349-00795-1,85:146,STI,Verkehrsbetriebe STI AG,Bus,85:146:1,1,10.0,B,...,False,8591839,"Spiez, Krone",01.04.2021 05:31,01.04.2021 05:31:12,PROGNOSE,01.04.2021 05:31,01.04.2021 05:31:12,PROGNOSE,False
87503,01.04.2021,85:146:256349-00795-1,85:146,STI,Verkehrsbetriebe STI AG,Bus,85:146:1,1,10.0,B,...,False,8576338,"Spiez, Oberlandstrasse",01.04.2021 05:32,01.04.2021 05:32:24,PROGNOSE,01.04.2021 05:32,01.04.2021 05:32:24,PROGNOSE,False
87504,01.04.2021,85:146:256349-00795-1,85:146,STI,Verkehrsbetriebe STI AG,Bus,85:146:1,1,10.0,B,...,False,8588105,"Spiez, Spiezmoos",01.04.2021 05:33,01.04.2021 05:33:54,PROGNOSE,01.04.2021 05:33,01.04.2021 05:33:54,PROGNOSE,False
87505,01.04.2021,85:146:256349-00795-1,85:146,STI,Verkehrsbetriebe STI AG,Bus,85:146:1,1,10.0,B,...,False,8588104,"Spiez, Kreuz",01.04.2021 05:35,01.04.2021 05:35:24,PROGNOSE,01.04.2021 05:35,01.04.2021 05:35:24,PROGNOSE,False


In [15]:
df.columns

Index(['BETRIEBSTAG', 'FAHRT_BEZEICHNER', 'BETREIBER_ID', 'BETREIBER_ABK',
       'BETREIBER_NAME', 'PRODUKT_ID', 'LINIEN_ID', 'LINIEN_TEXT', 'UMLAUF_ID',
       'VERKEHRSMITTEL_TEXT', 'ZUSATZFAHRT_TF', 'FAELLT_AUS_TF', 'BPUIC',
       'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'AN_PROGNOSE',
       'AN_PROGNOSE_STATUS', 'ABFAHRTSZEIT', 'AB_PROGNOSE',
       'AB_PROGNOSE_STATUS', 'DURCHFAHRT_TF'],
      dtype='object')

In [20]:
columns_eng = ['date', 'trip_id', 'company_id', 'company', 'company_name', 'product_id', 'trip_id', 'trip_type', 'round_trip_id', 'transport_type', 'is_replacement', 'is_cancelled', 'BPUIC', 'stop_station', 'arrival_time', 'arrival_pred', 'arrival_pred_status', 'departure_time', 'departure_pred', 'departure_pred_status', 'is_transit']
df.columns = columns_eng
df.head()

,date,trip_id,company_id,company,company_name,product_id,trip_id,trip_type,round_trip_id,transport_type,...,is_cancelled,BPUIC,stop_station,arrival_time,arrival_pred,arrival_pred_status,departure_time,departure_pred,departure_pred_status,is_transit
0,01.04.2021,80:06____:17001:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17001,RE,NaN,RE,...,False,8500090,Basel Bad Bf,01.04.2021 06:29,01.04.2021 06:30:00,PROGNOSE,NaN,NaN,PROGNOSE,False
1,01.04.2021,80:06____:17003:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17003,RE,NaN,RE,...,False,8500090,Basel Bad Bf,01.04.2021 08:02,01.04.2021 08:04:00,PROGNOSE,NaN,NaN,PROGNOSE,False
2,01.04.2021,80:06____:17005:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17005,RE,NaN,RE,...,True,8500090,Basel Bad Bf,01.04.2021 09:29,NaN,UNBEKANNT,NaN,NaN,UNBEKANNT,False
3,01.04.2021,80:06____:17006:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17006,RE,NaN,RE,...,False,8500090,Basel Bad Bf,NaN,NaN,PROGNOSE,01.04.2021 06:11,NaN,UNBEKANNT,False
4,01.04.2021,80:06____:17007:000,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,17007,RE,NaN,RE,...,True,8500090,Basel Bad Bf,01.04.2021 10:02,NaN,UNBEKANNT,NaN,NaN,UNBEKANNT,False


In [35]:
df.stop_station.sample(5)

1283143                Salmsach, Bodana
181962                   Lausanne, gare
1607990    Bellevue GE, chemin des Clys
158410                 Lausanne, Boston
1520618             GenÃ¿ve, ClaparÃ¿de
Name: stop_station, dtype: object

This file doesn't include departure stations, rendering it pretty much useless. Found the GTFS data and gonna work with that instead
___________
# correct data
Data [here](https://opentransportdata.swiss/dataset/1aff176a-9665-4395-a3b1-03e3032a0373/resource/05d05132-4fa6-44b0-a3d2-654d21a736ca/download/gtfs_fp2021_2021-03-31_09-10.zip)

Data is comma-separated text files, so pd.read_csv should do fine.

What we need from all the files:

from `stops.txt` the geographical location of each stop and its name
from `stop_times.txt` the arrival and departure times at each stop mentioned in `stops.txt`. The `trip_id` field can be used to identify stops belonging to a single trip

### HOW TO PROCEED:
- [ ] join the two tables and keep only the necessary columns
- [ ] Group the stops by trip to identify lines
- [ ] Identify connections (trips departing from a stop slightly after the arrival of another trip at the same stop)
- [ ] Build a connectivity graph
- [x] ???
- [ ] profit
